In [27]:
import torch 
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
words = open('names.txt', 'r').read().splitlines()
words[:3]

['emma', 'olivia', 'ava']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [63]:
block_size = 3
X, Y = [], []

for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [64]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [203]:
def build_dataset(words):
    block_size = 3
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random 
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182441, 3]) torch.Size([182441])
torch.Size([22902, 3]) torch.Size([22902])
torch.Size([22803, 3]) torch.Size([22803])


In [165]:
C = torch.randn((27, 2))

In [166]:
C[5]

tensor([0.0184, 0.0941])

In [167]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [168]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [169]:
h = torch.tanh((emb.view(-1, 6) @ W1) + b1)

In [170]:
h

tensor([[-0.9996, -0.6604,  0.9567,  ..., -0.5595, -0.9949, -0.9989],
        [-0.9994, -0.9977,  0.4674,  ..., -0.4661, -0.9995, -0.9997],
        [-0.6487, -0.7993,  1.0000,  ..., -0.9791, -0.9925,  0.9997],
        ...,
        [-0.9269, -0.9835, -0.5986,  ..., -0.8263,  0.7242, -0.9993],
        [ 0.2995, -0.4320,  0.7876,  ..., -0.9926,  0.9935, -0.9692],
        [-0.9776,  0.9982,  0.9657,  ..., -0.6925,  1.0000, -0.9830]])

In [171]:
h.shape

torch.Size([228146, 100])

In [172]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [173]:
logits = (h @ W2) + b2

In [174]:
logits.shape

torch.Size([228146, 27])

In [175]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)

In [176]:
prob.shape

torch.Size([228146, 27])

In [177]:
loss = -prob[torch.arange(prob.shape[0]), Y].log().mean()

In [178]:
loss

tensor(15.4671)

In [211]:
C = torch.randn((27, 2))
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
parameters = [C, W1, W2, b1, b2]

In [212]:
sum(p.nelement() for p in parameters)

3481

In [213]:
for p in parameters:
    p.requires_grad = True


In [214]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [220]:
lri = []
lossi = []
for i in range(30000):
    # making batches
    ix = torch.randint(0, Xtr.shape[0], (32,))
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    # print(loss.item())
    # backwards pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    lr = 0.01
    for p in parameters:
        p.data += -lr * p.grad

print(loss.item())


2.35361385345459


In [222]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.3294, grad_fn=<NllLossBackward0>)

In [223]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.3303, grad_fn=<NllLossBackward0>)

In [224]:
# training split, dev/validation split, test split
# 80%, 10%, 10%